# Композиции алгоритмов классификации

## Необходимые библиотеки

In [38]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [20]:
import warnings
warnings.filterwarnings('ignore')

## Подготовка данных

In [3]:
path_to_file = '../datasets/classification/occupancy_detection_preprocessed.csv'
dataset = pd.read_csv(path_to_file)

In [4]:
dataset.head()

,Unnamed: 0,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,140,2015-02-02 14:19:00,23.7000,26.272,585.200000,749.200000,0.004764,1
1,141,2015-02-02 14:19:59,23.7180,26.290,578.400000,760.400000,0.004773,1
2,142,2015-02-02 14:21:00,23.7300,26.230,572.666667,769.666667,0.004765,1
3,143,2015-02-02 14:22:00,23.7225,26.125,493.750000,774.750000,0.004744,1
4,144,2015-02-02 14:23:00,23.7540,26.200,488.600000,779.000000,0.004767,1


Удалим ненужные столбцы, разобьём выборку и отмасштабируем

In [5]:
dataset = dataset.drop(['Unnamed: 0', 'date'], axis=1)

In [6]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 5].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [8]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Композиции алгоритмов

В качестве базового алгоритма будем использовать **решающее дерево**. Сначала обучим его, измерив время. Затем обучим композиции алгоритмов с подбором гиперпараметров, измеряя время

### Базовый алгоритм

In [16]:
%%time
tree_clf = DecisionTreeClassifier()

params = {
    'max_depth': list(range(5, 11)),
    'min_samples_leaf': [5],
    'max_features': ['auto', 'sqrt', 'log2']
}

clf = GridSearchCV(tree_clf, params)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(clf.best_params_)

0.9905155642023347
{'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 5}
Wall time: 1.04 s


### Бэггинг

**Бэггинг**: каждый базовый аллгоритм обучается на подвыборке исходной обучающей выборки. Формирование подвыборке происходит при помощи *бутстрапа* (выбор с возвращением элементов из исходного множества)

In [24]:
%%time
bagging_clf = BaggingClassifier(DecisionTreeClassifier())

params = {
    'n_estimators': [10, 20, 30, 40, 50],
    'max_features': [0.8, 0.9, 1.0]
}

clf = GridSearchCV(bagging_clf, params)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(clf.best_params_)

0.9924610894941635
{'max_features': 0.8, 'n_estimators': 40}
Wall time: 43.9 s


### Стекинг

**Стекинг**: на основе обученных базовых алгоритмов строится новый *метаалгоритм*, для вывода предсказаний, основанных на множественных предсказаниях, возвращаемых этими базовыми алгоритмами

In [37]:
%%time
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svc', SVC(random_state=42))
]

stacking_clf = StackingClassifier(estimators, final_estimator=LogisticRegression(random_state=42))

stacking_clf.fit(X_train, y_train)

y_pred = stacking_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9931906614785992
Wall time: 3.49 s


### Градиентный бустинг

**Градиентный бустинг**: градиентный бустинг сводит задачу к градиентному спуску: на каждой итерации мы подгоняем базовый алгоритм к антиградиенту текущей ошибки подбора по отношению к текущей модели композиции

In [39]:
%%time
boosting_clf = GradientBoostingClassifier()

params = {
    'loss': ['deviance', 'exponential'],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200, 300]
}

clf = GridSearchCV(boosting_clf, params)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(clf.best_params_)

0.9914883268482491
{'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 300}
Wall time: 2min 49s


## Вывод

Благодаря композициям алгоритмов мы можем получить прирост в точности за счёт уменьшения влияния разброса в данных. Однако композиции дольше обучаются